In [1]:
import torch
import torchvision

from torchvision import models

import src.datasets.deep_fashion_ctsrbm

import os
import pathlib

import src.comps.backbones_cnn

from tqdm import tqdm

from torchsummary import summary

---

In [2]:
model = torchvision.models.convnext_tiny()

with open("misc/convnext_tiny_model_arch.txt", "w") as out_file:
    print(model, file=out_file)

In [10]:
ctsrbm_image_transform = torchvision.models.ConvNeXt_Tiny_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = src.datasets.deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform)

In [11]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for idx in range(8):
    model.features[idx].register_forward_hook(get_activation('features[{:d}]'.format(idx)))
model.features.register_forward_hook(get_activation('features'))
model.avgpool.register_forward_hook(get_activation('avgpool'))
model.classifier.register_forward_hook(get_activation('classifier'))

In [12]:
input = ctsrbm_dataset[0][0][None, :]
output = model(input)

In [13]:
print("{:20s} {:}".format("input", input.shape))
print("{:20s} {:}".format("output", output.shape))

print("---")

for key, item in activation.items():
    print("{:20s} {:}".format(key, item.shape))

input                torch.Size([1, 3, 224, 224])
output               torch.Size([1, 1000])
---
features[0]          torch.Size([1, 96, 56, 56])
features[1]          torch.Size([1, 96, 56, 56])
features[2]          torch.Size([1, 192, 28, 28])
features[3]          torch.Size([1, 192, 28, 28])
features[4]          torch.Size([1, 384, 14, 14])
features[5]          torch.Size([1, 384, 14, 14])
features[6]          torch.Size([1, 768, 7, 7])
features[7]          torch.Size([1, 768, 7, 7])
features             torch.Size([1, 768, 7, 7])
avgpool              torch.Size([1, 768, 1, 1])
classifier           torch.Size([1, 1000])


---

summary(model, input_size=(3, 224, 224), device="cpu")

---

In [3]:
model = torchvision.models.convnext_tiny()

ctsrbm_image_transform = torchvision.models.ConvNeXt_Tiny_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = src.datasets.deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, ctsrbm_image_transform)

In [4]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for idx in [1, 3, 5, 7]:
    model.features[idx].register_forward_hook(get_activation('features[{:d}]'.format(idx)))

In [5]:
size_list = list(range(32, 512))
feature_1_shape_list = []
feature_3_shape_list = []
feature_5_shape_list = []
feature_7_shape_list = []

for size in tqdm(size_list):

    input = ctsrbm_dataset[0][0][None, :, :size, :size]
    output = model(input)

    feature_1_shape_list.append(tuple(activation["features[1]"].shape))
    feature_3_shape_list.append(tuple(activation["features[3]"].shape))
    feature_5_shape_list.append(tuple(activation["features[5]"].shape))
    feature_7_shape_list.append(tuple(activation["features[7]"].shape))

100%|██████████| 480/480 [00:17<00:00, 27.71it/s]


In [6]:
last_perm_size = 0

print("Img size | Ftr size 1")

for idx, size in enumerate(size_list):

    perm_size = feature_1_shape_list[idx][-1]

    if last_perm_size < perm_size:
        print("     {:3d} |        {:3d}".format(size, perm_size))
        last_perm_size = perm_size

Img size | Ftr size 1
      32 |          8
      36 |          9
      40 |         10
      44 |         11
      48 |         12
      52 |         13
      56 |         14
      60 |         15
      64 |         16
      68 |         17
      72 |         18
      76 |         19
      80 |         20
      84 |         21
      88 |         22
      92 |         23
      96 |         24
     100 |         25
     104 |         26
     108 |         27
     112 |         28
     116 |         29
     120 |         30
     124 |         31
     128 |         32
     132 |         33
     136 |         34
     140 |         35
     144 |         36
     148 |         37
     152 |         38
     156 |         39
     160 |         40
     164 |         41
     168 |         42
     172 |         43
     176 |         44
     180 |         45
     184 |         46
     188 |         47
     192 |         48
     196 |         49
     200 |         50
     204 |         51
     208 |

In [7]:
last_perm_size = 0

print("Img size | Ftr size 3")

for idx, size in enumerate(size_list):

    perm_size = feature_3_shape_list[idx][-1]

    if last_perm_size < perm_size:
        print("     {:3d} |        {:3d}".format(size, perm_size))
        last_perm_size = perm_size

Img size | Ftr size 3
      32 |          4
      40 |          5
      48 |          6
      56 |          7
      64 |          8
      72 |          9
      80 |         10
      88 |         11
      96 |         12
     104 |         13
     112 |         14
     120 |         15
     128 |         16
     136 |         17
     144 |         18
     152 |         19
     160 |         20
     168 |         21
     176 |         22
     184 |         23
     192 |         24
     200 |         25
     208 |         26
     216 |         27
     224 |         28


In [8]:
last_perm_size = 0

print("Img size | Ftr size 5")

for idx, size in enumerate(size_list):

    perm_size = feature_5_shape_list[idx][-1]

    if last_perm_size < perm_size:
        print("     {:3d} |        {:3d}".format(size, perm_size))
        last_perm_size = perm_size

Img size | Ftr size 5
      32 |          2
      48 |          3
      64 |          4
      80 |          5
      96 |          6
     112 |          7
     128 |          8
     144 |          9
     160 |         10
     176 |         11
     192 |         12
     208 |         13
     224 |         14


In [9]:
last_perm_size = 0

print("Img size | Ftr size 7")

for idx, size in enumerate(size_list):

    perm_size = feature_7_shape_list[idx][-1]

    if last_perm_size < perm_size:
        print("     {:3d} |        {:3d}".format(size, perm_size))
        last_perm_size = perm_size

Img size | Ftr size 7
      32 |          1
      64 |          2
      96 |          3
     128 |          4
     160 |          5
     192 |          6
     224 |          7


In [12]:
def feature_1_shape_formula(size):
    return min(size // 4, 56)

for idx, size in enumerate(size_list):

    perm_size = feature_1_shape_list[idx][-1]
    pred_perm_size = feature_1_shape_formula(size)

    if perm_size != pred_perm_size:
        print("ERROR: size = {:4d} | perm_size = {:4d} | pred_perm_size = {:4d}".format(
            size,
            perm_size,
            pred_perm_size
        ))

In [13]:
def feature_3_shape_formula(size):
    return min(size // 8, 28)

for idx, size in enumerate(size_list):

    perm_size = feature_3_shape_list[idx][-1]
    pred_perm_size = feature_3_shape_formula(size)

    if perm_size != pred_perm_size:
        print("ERROR: size = {:4d} | perm_size = {:4d} | pred_perm_size = {:4d}".format(
            size,
            perm_size,
            pred_perm_size
        ))

In [14]:
def feature_5_shape_formula(size):
    return min(size // 16, 14)

for idx, size in enumerate(size_list):

    perm_size = feature_5_shape_list[idx][-1]
    pred_perm_size = feature_5_shape_formula(size)

    if perm_size != pred_perm_size:
        print("ERROR: size = {:4d} | perm_size = {:4d} | pred_perm_size = {:4d}".format(
            size,
            perm_size,
            pred_perm_size
        ))

In [15]:
def feature_7_shape_formula(size):
    return min(size // 32, 7)

for idx, size in enumerate(size_list):

    perm_size = feature_7_shape_list[idx][-1]
    pred_perm_size = feature_7_shape_formula(size)

    if perm_size != pred_perm_size:
        print("ERROR: size = {:4d} | perm_size = {:4d} | pred_perm_size = {:4d}".format(
            size,
            perm_size,
            pred_perm_size
        ))